# 🤖 AEC Compliance Agent - Clean Display Version

## 🎯 Learning Goals
- **Understand** how the ReAct agent works with building data
- **Learn** to use the 7 essential compliance tools
- **See** clean tool usage summaries (max 2 lines + "...")
- **Explore** autonomous compliance verification

## 🚀 What You'll Learn
1. **Agent Setup** - Environment, data loading, RAG connection
2. **Tool Usage** - Clean display of tool calls and results
3. **Autonomous Analysis** - Agent reasoning and decision making
4. **Real Building Data** - Working with actual Vilamalla project

---


In [9]:
import sys
from pathlib import Path

# Load environment variables from .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ Environment variables loaded from .env file")
except ImportError:
    print("⚠️ python-dotenv not installed - please install with: pip install python-dotenv")
    print("   Or manually set OPENAI_API_KEY in your environment")

# Add project root to Python path
ROOT = Path.cwd()
while ROOT != ROOT.parent and not (ROOT / 'src').exists():
    ROOT = ROOT.parent

if (ROOT / 'src').exists():
    sys.path.insert(0, str(ROOT))
    print(f"✅ Project root found: {ROOT}")
else:
    sys.path.append('.')
    print(f"✅ Using current directory: {Path.cwd()}")

# Import required modules
try:
    from langchain_core.messages import HumanMessage
    from src.agent.graph import create_compliance_agent
    from src.agent.tools import load_project_data, set_vectorstore_manager
    print("✅ Core imports successful")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("   Please ensure you're running from the project directory")
    print("   and all dependencies are installed")
    raise

# Check for OPENAI_API_KEY
import os
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("✅ OPENAI_API_KEY found in environment")
else:
    print("⚠️ OPENAI_API_KEY not found in environment")
    print("   Agent creation will fail without API key")
    print("   Please set your OpenAI API key in .env file")


✅ Environment variables loaded from .env file
✅ Project root found: /Users/rauladell/Work/Servitec/aec-compliance-agent
✅ Core imports successful
✅ OPENAI_API_KEY found in environment


In [10]:
# Load project data and create agent
project_file = ROOT / "data/extracted/vilamalla_building.json"
project_data_available = False

if project_file.exists():
    try:
        project_data = load_project_data(project_file)
        project_data_available = True
        print(f"✅ Real Vilamalla building data loaded: {project_file}")
        
        doors = project_data.get_all_doors()
        print(f"   • {len(doors)} doors available")
        if doors:
            print(f"   • Sample door: {doors[0].name}, width: {doors[0].width_mm}mm")
        
    except Exception as e:
        print(f"⚠️ Error loading project data: {e}")
        project_data_available = False
else:
    print(f"ℹ️ Vilamalla data not found: {project_file}")

# Create agent
agent = None
agent_ready = False

if api_key:
    try:
        agent = create_compliance_agent()
        print("✅ LangGraph agent created successfully")
        
        # Test basic functionality
        test_result = agent.invoke({
            "messages": [HumanMessage(content="Hello, introduce yourself in one sentence.")],
            "max_iterations": 1
        })
        
        if test_result and test_result.get("iterations", 0) >= 1:
            agent_ready = True
            print("✅ LangGraph agent basic functionality verified")
        else:
            print("⚠️ Agent created but basic test failed")
            
    except Exception as e:
        print(f"❌ Error creating or testing agent: {e}")
        agent = None
else:
    print("❌ Cannot create agent without OPENAI_API_KEY")

# Import clean display helper
from agent_display_helper import run_agent_tools_only, run_agent_with_clean_display
print("✅ Clean display helper imported")


✅ Real Vilamalla building data loaded: /Users/rauladell/Work/Servitec/aec-compliance-agent/data/extracted/vilamalla_building.json
   • 23 doors available
   • Sample door: PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3349373, width: 900.0mm
✅ LangGraph agent created successfully
✅ LangGraph agent basic functionality verified
✅ Clean display helper imported


## 🧠 RAG Integration Explanation

**The agent uses the same RAG system from Notebook 3:**

- ✅ **ChromaDB Vectorstore** - Contains CTE DB-SI and DB-SUA regulations
- ✅ **Semantic Search** - Finds relevant regulation sections automatically  
- ✅ **Context-Aware** - Applies regulations to specific building geometry
- ✅ **Tool Integration** - Combines RAG queries with geometric analysis

**How it works:**
1. **Agent receives prompt** → Identifies compliance requirements
2. **RAG queries CTE** → Retrieves relevant regulation text
3. **Tools analyze geometry** → Checks building against regulations
4. **Agent synthesizes** → Provides compliance assessment + recommendations

**Example RAG queries the agent will make:**
- "CTE DB-SI 1 fire compartment continuity requirements"
- "CTE DB-SUA 9 door accessibility minimum widths"
- "CTE DB-SI 3 evacuation distance limits"
- "CTE DB-SI 6 structural continuity requirements"

---


## 📋 Quick Reference

### 🎯 **Available Demo Cells:**
- **Demo 1**: Basic tool usage + agent response
- **Demo 2**: DB-SI 1 - Fire compartment continuity
- **Demo 3**: DB-SI 3 - Evacuation paths & distances  
- **Demo 4**: DB-SUA 9 - Door clearance & accessibility
- **Demo 5**: DB-SI 6 - Structural continuity
- **Demo 6**: Comprehensive CTE compliance analysis

### 🔧 **Key Functions:**
```python
# Clean tool usage + final response
run_agent_tools_only(agent, prompt, max_iterations=8)

# Full conversation display
run_agent_with_clean_display(agent, prompt, max_iterations=8, title="ANALYSIS")
```

### 🧠 **RAG + Tools Integration:**
- **RAG queries** CTE regulations automatically
- **Tools analyze** building geometry  
- **Agent synthesizes** compliance assessment
- **Clean display** shows tool usage + final response

**Ready for comprehensive CTE compliance verification!** 🎯✨


# 🤖 Tutorial 4: ReAct Agent - Autonomous Compliance Verification

## 🎯 What You'll Learn
- What is a ReAct agent
- How agents use tools autonomously
- See real compliance verification
- Watch autonomous reasoning

---

## 🛠️ Essential Tools (Simplified)

Our agent now has **7 focused tools** for maximum clarity and efficiency:

**Data & Building Information:**
1. **extract_ifc_data(ifc_file_path, output_path)** - Extract building data from IFC files
2. **list_all_doors()** - List all doors in the project

**Core Geometric Analysis (3 of 4 core functions):**
3. **create_circulation_graph_tool()** - Create circulation graph for pathfinding and connectivity
4. **find_nearest_door_tool(point_x, point_y)** - Find nearest door to any point
5. **calculate_clearance_tool(elem1_type, elem1_id, elem2_type, elem2_id)** - Calculate clearance between elements

**Compliance Verification:**
6. **check_door_width_compliance(door_id)** - Check if a door meets width requirements
7. **query_normativa(question)** - Query building codes via RAG

**Key Benefits:**
- ✅ **Simple & Focused**: Only essential tools for core workflow
- ✅ **3 Core Functions**: Connectivity, proximity, clearance calculations  
- ✅ **End-to-End**: IFC → Analysis → Compliance verification
- ✅ **RAG Integration**: Real building code queries

## 🚀 Setup: Environment & Agent Creation

This cell loads the environment, imports required modules, loads the real Vilamalla building data, sets up the RAG system, and creates the LangGraph agent.

In [4]:
import sys
from pathlib import Path

# Load environment variables from .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ Environment variables loaded from .env file")
except ImportError:
    print("⚠️ python-dotenv not installed - please install with: pip install python-dotenv")
    print("   Or manually set OPENAI_API_KEY in your environment")

# Add project root to Python path
ROOT = Path.cwd()
while ROOT != ROOT.parent and not (ROOT / 'src').exists():
    ROOT = ROOT.parent

if (ROOT / 'src').exists():
    sys.path.insert(0, str(ROOT))
    print(f"✅ Project root found: {ROOT}")
else:
    sys.path.append('.')
    print(f"✅ Using current directory: {Path.cwd()}")

# Import required modules
try:
    from langchain_core.messages import HumanMessage
    from src.agent.graph import create_compliance_agent
    from src.agent.tools import load_project_data, set_vectorstore_manager
    print("✅ Core imports successful")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("   Please ensure you're running from the project directory")
    print("   and all dependencies are installed")
    raise

# Check for OPENAI_API_KEY
import os
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print("✅ OPENAI_API_KEY found in environment")
else:
    print("⚠️ OPENAI_API_KEY not found in environment")
    print("   Agent creation will fail without API key")
    print("   Please set your OpenAI API key in .env file")

# Load project data using the SAME approach as notebooks 01-02
project_file = ROOT / "data/extracted/vilamalla_building.json"
project_data_available = False

if project_file.exists():
    try:
        project_data = load_project_data(project_file)
        project_data_available = True
        print(f"✅ Real Vilamalla building data loaded: {project_file}")
        
        # Show some detailed stats
        doors = project_data.get_all_doors()
        rooms = project_data.get_all_rooms()
        walls = []
        for level in project_data.levels:
            walls.extend(level.walls)
            
        print(f"   • {len(project_data.levels)} levels")
        print(f"   • {len(rooms)} rooms total")
        print(f"   • {len(doors)} doors total")
        print(f"   • {len(walls)} walls total")
        
        if doors:
            sample_door = doors[0]
            print(f"   • Sample door: {sample_door.name}, width: {sample_door.width_mm}mm")
        
        if walls:
            sample_wall = walls[0]
            print(f"   • Sample wall: {sample_wall.id}, thickness: {sample_wall.thickness_mm}mm, height: {sample_wall.height_mm}mm")
        
    except Exception as e:
        print(f"⚠️ Error loading project data: {e}")
        print("   Agent will work but some tools may not function optimally")
        project_data_available = False
else:
    print(f"ℹ️ Vilamalla data not found: {project_file}")
    print("   Agent will work with any valid project data")

# Setup RAG vectorstore (use exact same approach as notebook 03)
rag_available = False
vectorstore_path = ROOT / "vectorstore/normativa_db"

print("📦 RAG SETUP (matching notebook 03 approach):")
print(f"   Vectorstore path: {vectorstore_path}")
print(f"   Vectorstore exists: {vectorstore_path.exists()}")

try:
    # Use the same approach as notebook 03 - direct ChromaDB access
    import chromadb
    from sentence_transformers import SentenceTransformer  # may be unused depending on your tools
    from langchain_core.documents import Document          # may be unused depending on your tools

    client = chromadb.PersistentClient(path=str(vectorstore_path))
    collection = client.get_collection("langchain")
    
    # Test that it works
    test_result = collection.query(query_texts=["test"], n_results=1)
    print(f"✅ ChromaDB direct access successful: {len(test_result['documents'][0])} docs found")

    class ChromaDBRAGWrapper:
        def __init__(self, collection):
            self.collection = collection
        
        def query(self, question):
            try:
                result = self.collection.query(query_texts=[question], n_results=3)
                docs = result['documents'][0] if result['documents'] else []
                
                if docs:
                    # Use first 2 documents for context
                    context = ' '.join(docs[:2])
                    return {
                        'answer': (
                            "Según la normativa CTE: "
                            f"{context[:400]}... "
                            "(Respuesta basada en documentos recuperados del vectorstore de notebook 03)"
                        )
                    }
                else:
                    return {
                        'answer': (
                            f"No se encontró información específica para: {question}. "
                            "Consulte el CTE DB-SI para requisitos detallados."
                        )
                    }
            except Exception:
                return {
                    'answer': (
                        f"Error consultando normativa para: {question}. "
                        "Revise la documentación CTE manualmente."
                    )
                }

    set_vectorstore_manager(ChromaDBRAGWrapper(collection))
    rag_available = True
    print("✅ RAG system connected (using same vectorstore as notebook 03)")
    
except Exception as e:
    print(f"⚠️ RAG setup failed: {e}")
    print("   CTE regulation queries will use basic fallback responses")
    rag_available = False

# Create agent with fixed LangGraph implementation
agent = None
agent_ready = False

if api_key:
    try:
        agent = create_compliance_agent()
        print("✅ LangGraph agent created successfully")
        
        # Test with a very simple non-tool query first
        test_result = agent.invoke({
            "messages": [HumanMessage(content="Hello, introduce yourself in one sentence.")],
            "max_iterations": 1
        })
        
        if test_result and test_result.get("iterations", 0) >= 1:
            agent_ready = True
            print("✅ LangGraph agent basic functionality verified")
        else:
            print("⚠️ Agent created but basic test failed")
            
    except Exception as e:
        print(f"❌ Error creating or testing agent: {e}")
        agent = None
else:
    print("❌ Cannot create agent without OPENAI_API_KEY")

# Final status
if agent and agent_ready:
    print("\n🤖 Ready to verify compliance with real Vilamalla building data!")
    if rag_available and project_data_available:
        print("   • LangGraph agent created and tested successfully")
        print("   • RAG system connected (same vectorstore as notebook 03)")
        print("   • 7 essential tools available")
        print("   • Using real building data from notebooks 01-02")
        print("   • Schema compatibility issues FIXED (field transformation handled internally)")
        print("   • Data transformation applied for robust schema handling")
    else:
        print("   • LangGraph agent created and tested successfully")
        status = []
        if not rag_available:
            status.append("RAG queries will use basic fallback responses")
        if not project_data_available:
            status.append("Limited building data access")
        if status:
            print(f"   • {', '.join(status)}")
        print("   • 7 essential tools available")
    
    print("\n💡 FIXES APPLIED:")
    print("   ✓ LangGraph tool execution flow corrected")
    print("   ✓ Data loading uses internal transformation (width->width_mm, etc.)")
    print("   ✓ Field transformation for schema compatibility")
    print("   ✓ Real Vilamalla building data integration")
    
elif agent:
    print("\n⚠️ Agent created but not fully functional")
    print("   Basic interactions work, but tool execution may have issues")
    
else:
    print("\n⚠️ Agent creation failed - notebook demos will not work")
    print("   Please check your environment setup and try again")


✅ Environment variables loaded from .env file
✅ Project root found: /Users/rauladell/Work/Servitec/aec-compliance-agent
✅ Core imports successful
✅ OPENAI_API_KEY found in environment
✅ Real Vilamalla building data loaded: /Users/rauladell/Work/Servitec/aec-compliance-agent/data/extracted/vilamalla_building.json
   • 9 levels
   • 9 rooms total
   • 23 doors total
   • 102 walls total
   • Sample door: PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3349373, width: 900.0mm
   • Sample wall: W532, thickness: 200.0mm, height: 2700.0mm
📦 RAG SETUP (matching notebook 03 approach):
   Vectorstore path: /Users/rauladell/Work/Servitec/aec-compliance-agent/vectorstore/normativa_db
   Vectorstore exists: True
✅ ChromaDB direct access successful: 1 docs found
✅ RAG system connected (using same vectorstore as notebook 03)
✅ LangGraph agent created successfully
✅ LangGraph agent basic functionality verified

🤖 Ready to verify compliance with real Vilamalla building data!
   • LangGraph agent created and te

## 🗺️ Real Building Data - Vilamalla Project

We're now using the **real Vilamalla building data** from notebooks 01-02 (`data/extracted/vilamalla_building.json`).

What it contains:
- **9 levels** with actual building structure
- **Real doors** (e.g., "PUERTA IKEMA BASICA1" with 900mm width)  
- **Rooms with boundaries** and proper spatial data
- **Walls** with concrete materials and fire ratings
- **Real coordinates** from the original IFC file

This is the same dataset we've been exploring in the previous tutorials, giving the agent authentic building data to analyze.


## 🎯 Clean Agent Display - Tool Usage Summary

### ✨ **Problem Solved!**

Instead of seeing verbose tool logs like this:
```
ToolMessage: [{"id": "D3283", "name": "PUERTA IKEMA BASICA1:PI 3_5000x5000 mm:3349373", "width_mm": 900.0, "door_type": "single", "from_room": null, "to_room": null, "is_emergency_exit": false, "fire_rating": null}, ...]
```

You now see clean summaries like this:
```
🔧 TOOLS USED:
   🔧 list_all_doors: 📋 23 doors (D3283)
   🔧 check_door_width_compliance: 🔍 D3283: COMPLIANT
   🔧 create_circulation_graph_tool: ✅ Created circulation graph with 9 rooms a...
```

### 🚀 **How to Use**

**Option 1: Tool Usage Only (Minimal)**
```python
from agent_display_helper import run_agent_tools_only

result = run_agent_tools_only(
    agent, 
    "Check door compliance for the first 3 doors",
    max_iterations=5
)
```

**Option 2: Full Clean Display**
```python
from agent_display_helper import run_agent_with_clean_display

result = run_agent_with_clean_display(
    agent,
    "Analyze building circulation and connectivity",
    max_iterations=8,
    title="CIRCULATION ANALYSIS"
)
```

### 🎯 **Benefits**

- ✅ **Clean Output**: No more verbose JSON dumps
- ✅ **Quick Overview**: See exactly which tools were used
- ✅ **Summarized Results**: Key information in 1-2 lines max
- ✅ **Professional Look**: Clean, readable format
- ✅ **Flexible**: Choose minimal or full display


## 🔧 Demo 1: Simple Tool Usage (Clean Display)

This demo shows the **minimal tool usage display** - you only see which tools were used and a brief summary of their results.


In [6]:
# Demo 1: Simple Tool Usage (Clean Display)
print("🔧 DEMO 1: Clean Tool Usage Summary")
print("="*50)

if agent is not None:
    from agent_display_helper import run_agent_tools_only
    
    # Example: Check door compliance
    result = run_agent_tools_only(
        agent, 
        "IMPORTANT: Building data is already loaded. Check door compliance for doors D3283 and D3379. What size are they?",
        max_iterations=5
    )
    
    print("\n✅ Perfect! Clean, concise tool usage display!")
    
else:
    print("❌ Agent not available - please check the setup cell above")


🔧 DEMO 1: Clean Tool Usage Summary
🔧 TOOLS USED:
   🔧 check_door_width_compliance: 🔍 D3283: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D3379: COMPLIANT

✅ Completed in 3 iterations

🤖 AGENT RESPONSE:
Both doors, D3283 and D3379, are compliant with the minimum width requirement of 800mm. They have a clear width of 850mm, meeting the regulatory standards specified in CTE DB-SUA Section 2.1.

✅ Perfect! Clean, concise tool usage display!


## 🔧 Demo 2: Full Clean Display

This demo shows the **full clean display** - you see the conversation flow plus clean tool summaries.


In [7]:
# Demo 2: Full Clean Display
print("🔧 DEMO 2: Full Clean Display")
print("="*50)

if agent is not None:
    from agent_display_helper import run_agent_with_clean_display
    
    # Example: Analyze building circulation
    result = run_agent_with_clean_display(
        agent,
        "IMPORTANT: Building data is already loaded. Analyze building circulation and check connectivity",
        max_iterations=8,
        title="CIRCULATION ANALYSIS"
    )
    
    print("\n✅ Perfect! Clean conversation + tool summaries!")
    
else:
    print("❌ Agent not available - please check the setup cell above")


🔧 DEMO 2: Full Clean Display
🤖 CIRCULATION ANALYSIS

🤖 AGENT CONVERSATION:
--------------------------------------------------
👤 USER: IMPORTANT: Building data is already loaded. Analyze building circulation and check connectivity

🤖 AGENT: The circulation graph analysis reveals the following information:
- Total rooms: 9
- Total connections: 0
- Exit rooms: 0
- Connected status: No
- Average path length: Not available
- Connectivity score: 0.0

The building does not have any connections between rooms, indicating a lack of circulation flow. This could impact accessibility and functionality within the building. Further analysis and corrective measures may be necessary to improve connectivity. Let's proceed with further checks to assess the building's compliance and connectivity.

🤖 AGENT: I have listed all the doors in the project along with their basic information. Here are some key details about the doors:
- Total number of doors: 23
- Door width: 900.0 mm
- Door type: Single
- Emergen

## 🧠 Autonomous Compliance Agent

This agent makes **its own decisions** about what to check and how to verify compliance. It's guided by general principles but **not constrained** to follow rigid steps.

### 🎯 **Agent Guidance Principles:**
- **Discover**: Understand the building by exploring the data
- **Reason**: Identify potential compliance issues based on findings
- **Verify**: Check regulations and calculate compliance
- **Recommend**: Provide actionable insights

### 🤖 **Agent Autonomy:**
- **Chooses its own analysis path** based on its findings
- **Chooses tools** in any order that makes logical sense
- **Follows its own reasoning** rather than rigid checklists
- **Adapts approach** based on building characteristics


## 🕵️ Demo 3: Autonomous Compliance Analysis

This demo shows the agent **making its own decisions** about what to investigate and how to verify compliance.


In [8]:
# Demo 3: Autonomous Compliance Analysis
print("🕵️ DEMO 3: Autonomous Compliance Analysis")
print("="*50)

if agent is not None:
    from agent_display_helper import run_agent_with_clean_display
    
    # Example: Give agent complete freedom to analyze
    result = run_agent_with_clean_display(
        agent,
        """
IMPORTANT: The building data is already loaded from JSON - you do NOT need to extract from IFC files.

You are a compliance expert analyzing this building. I want you to:

1. First, explore the building data using list_all_doors() to understand what you're working with
2. Use your professional judgment to identify what compliance aspects are most important to check
3. Choose your own analysis approach - use tools like check_door_width_compliance(), create_circulation_graph_tool(), or query_normativa()
4. If you find something concerning, investigate it further using appropriate tools
5. Provide insights and recommendations based on your findings

You have complete freedom to decide what to analyze and how. Focus on what matters most for building safety and compliance.

Start by listing the doors to get familiar with the building, then follow your instincts about what needs to be checked.
""",
        max_iterations=15,
        title="AUTONOMOUS COMPLIANCE ANALYSIS"
    )
    
    print("\n✅ Agent made its own decisions about what to check and how!")
    
else:
    print("❌ Agent not available - please check the setup cell above")


🕵️ DEMO 3: Autonomous Compliance Analysis
🤖 AUTONOMOUS COMPLIANCE ANALYSIS

🤖 AGENT CONVERSATION:
--------------------------------------------------
👤 USER: 
IMPORTANT: The building data is already loaded from JSON - you do NOT need to extract from IFC files.

You are a compliance expert analyzing this building. I want you to:

1. First, explore the building data using list_all_doors() to understand what you're working with
2. Use your professional judgment to identify what compliance aspects are most important to check
3. Choose your own analysis approach - use tools like check_door_width_compliance(), create_circulation_graph_tool(), or query_normativa()
4. If you find something concerning, investigate it further using appropriate tools
5. Provide insights and recommendations based on your findings

You have complete freedom to decide what to analyze and how. Focus on what matters most for building safety and compliance.

Start by listing the doors to get familiar with the building, 

In [18]:
# Demo 4: DB-SI 1 - Fire Compartment Continuity & Wall Gaps
if agent and agent_ready:
    print("🔥 DEMO 2: DB-SI 1 - Fire Compartment Continuity")
    print("=" * 60)
    
    result = run_agent_tools_only(
        agent, 
        "IMPORTANT: Building data is already loaded. Analyze fire compartment continuity according to CTE DB-SI 1.",
        max_iterations=10
    )
    
    print("\n✅ Fire compartment continuity analysis completed!")
else:
    print("❌ Agent not ready - please run setup cells first")


🔥 DEMO 2: DB-SI 1 - Fire Compartment Continuity
🔧 TOOLS USED:
   🔧 create_circulation_graph_tool: ✅ Created circulation graph with 9 rooms a...

✅ Completed in 2 iterations

🤖 AGENT RESPONSE:
The circulation graph has been created with 9 rooms and 0 connections. The analysis indicates that the rooms are not connected. This lack of connectivity could impact fire compartment continuity according to CTE DB-SI 1. 

Next steps:
1. Identify the rooms that are not connected.
2. Determine the impact of this lack of connectivity on fire compartment continuity.
3. Check the compliance of the building elements within the rooms to ensure fire safety standards are met.

✅ Fire compartment continuity analysis completed!


In [21]:
# Demo 5: DB-SI 6 - Structural Continuity (Geometry-based)
if agent and agent_ready:
    print("🏗️ DEMO 5: DB-SI 6 - Structural Continuity")
    print("=" * 60)
    
    result = run_agent_tools_only(
        agent, 
        "IMPORTANT: Building data is already loaded. Check if Structural Continuity is compliant according to the DB-SI.",
        max_iterations=30
    )
    
    print("\n✅ Structural continuity analysis completed!")
else:
    print("❌ Agent not ready - please run setup cells first")


🏗️ DEMO 5: DB-SI 6 - Structural Continuity
🔧 TOOLS USED:
   🔧 query_normativa: 📋 Según la normativa CTE: se a tener más d...

✅ Completed in 2 iterations

🤖 AGENT RESPONSE:
Based on the CTE regulations, for buildings with more than 12 dwellings on floors without accessible main entrances, compliance with the DB-SI is required. Additionally, when a building undergoes an increase in evacuation height, it must conform to all DB-SI conditions associated with the new height.

To ensure compliance with Structural Continuity according to the DB-SI, the building must be adapted to meet the conditions related to the new evacuation height. It is essential to verify and ensure effective adaptation without worsening the conditions during changes of use in existing buildings.

✅ Structural continuity analysis completed!


In [22]:
# Demo 6: DB-SI 3 - Evacuation Paths & Distances
if agent and agent_ready:
    print("🚪 DEMO 3: DB-SI 3 - Evacuation Paths & Distances")
    print("=" * 60)
    
    result = run_agent_tools_only(
        agent, 
        "IMPORTANT: Building data is already loaded. Analyze evacuation paths and check if they are compliant.",
        max_iterations=10
    )
    
    print("\n✅ Evacuation path analysis completed!")
else:
    print("❌ Agent not ready - please run setup cells first")

🚪 DEMO 3: DB-SI 3 - Evacuation Paths & Distances
🔧 TOOLS USED:
   🔧 create_circulation_graph_tool: ✅ Created circulation graph with 9 rooms a...
   🔧 list_all_doors: 📋 23 doors (D3283)
   🔧 check_door_width_compliance: 🔍 D3283: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D3379: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D3392: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D3405: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D3418: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D3431: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D3442: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D3453: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D3464: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D3475: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D3486: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D5169: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D5180: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D6630: COMPLIANT
   🔧 check_door_width_compliance: 🔍 D6641: COMPLIANT
   🔧 check_door_widt